In [2]:
from pyspark.sql import SparkSession


In [3]:
#TODO: make a method out of this.

import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk1.8.0_241.jdk/Contents/Home'

import subprocess
subprocess.check_output(['java', '-version'], stderr=subprocess.STDOUT)

b'java version "1.8.0_241"\nJava(TM) SE Runtime Environment (build 1.8.0_241-b07)\nJava HotSpot(TM) 64-Bit Server VM (build 25.241-b07, mixed mode)\n'

In [4]:
spark = SparkSession.builder.appName("Basics").getOrCreate()
df = spark.read.parquet("data/final_join.parquet")


In [5]:
df.show()

+--------------------+--------------------------+---------+---------+----------+------+----------+--------------------+---------------+-----------------+----------+--------------------+-----------------------------+
|               email|creationDate_comment_month|person_id|firstName|  lastName|gender|  birthday| creationDate_person|     locationIP|      browserUsed|comment_id|creationDate_comment|Number_of_comments_that_month|
+--------------------+--------------------------+---------+---------+----------+------+----------+--------------------+---------------+-----------------+----------+--------------------+-----------------------------+
|David219902326268...|                         9|     7134|    David|    Alonso|female|1983-10-29|2019-03-20 04:12:...| 31.222.107.200|           Chrome|   2123877|2019-09-24 07:27:...|                            3|
|David219902326407...|                         5|     8523|    David|   Vasquez|female|1988-08-18|2019-03-13 23:09:...|  200.10.250.73| 

First we'll drop some columns and impute some others and after that we'll do some basic data analysis, 
largely based on this post: https://medium.com/@aieeshashafique/exploratory-data-analysis-using-pyspark-dataframe-in-python-bd55c02a2852
(part1, this file)

Then We'll do some more advanced EDA, 
after which we'll choose a cutoff value which will allow us to define "an inactive user" (part 2)

In [6]:
#dropping first- and lastname columns
df = df.select([c for c in df.columns if c not in {'lastName','firstName'}])

# Adding a lot of datetime columns
from pyspark.sql.functions import dayofmonth,dayofweek,dayofyear,year
columns = [dayofmonth,dayofweek,dayofyear,year]
for x in columns:
    df = df.withColumn(f"creationDate_comment_{x.__name__}", x(df['creationDate_comment']))

# extracting domain name from emails
from pyspark.sql.functions import split
df = df.withColumn('email', split(df['email'], '@')[1])

#Hardcoding the year in is not ideal, but for our purposes allowable 
df = df.withColumn('age', 2020 - year(df['birthday'])).drop('birthday')


In [7]:
df.show()

+-----------+--------------------------+---------+------+--------------------+---------------+-----------------+----------+--------------------+-----------------------------+-------------------------------+------------------------------+------------------------------+-------------------------+---+
|      email|creationDate_comment_month|person_id|gender| creationDate_person|     locationIP|      browserUsed|comment_id|creationDate_comment|Number_of_comments_that_month|creationDate_comment_dayofmonth|creationDate_comment_dayofweek|creationDate_comment_dayofyear|creationDate_comment_year|age|
+-----------+--------------------------+---------+------+--------------------+---------------+-----------------+----------+--------------------+-----------------------------+-------------------------------+------------------------------+------------------------------+-------------------------+---+
|  gmail.com|                         9|     7134|female|2019-03-20 04:12:...| 31.222.107.200|         

In [8]:
# 1.schema of pyspark dataframe

df.printSchema()


root
 |-- email: string (nullable = true)
 |-- creationDate_comment_month: integer (nullable = true)
 |-- person_id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- creationDate_person: timestamp (nullable = true)
 |-- locationIP: string (nullable = true)
 |-- browserUsed: string (nullable = true)
 |-- comment_id: integer (nullable = true)
 |-- creationDate_comment: timestamp (nullable = true)
 |-- Number_of_comments_that_month: long (nullable = true)
 |-- creationDate_comment_dayofmonth: integer (nullable = true)
 |-- creationDate_comment_dayofweek: integer (nullable = true)
 |-- creationDate_comment_dayofyear: integer (nullable = true)
 |-- creationDate_comment_year: integer (nullable = true)
 |-- age: integer (nullable = true)



In [9]:
# 2.Show your PySpark Dataframe 

df.show()

+-----------+--------------------------+---------+------+--------------------+---------------+-----------------+----------+--------------------+-----------------------------+-------------------------------+------------------------------+------------------------------+-------------------------+---+
|      email|creationDate_comment_month|person_id|gender| creationDate_person|     locationIP|      browserUsed|comment_id|creationDate_comment|Number_of_comments_that_month|creationDate_comment_dayofmonth|creationDate_comment_dayofweek|creationDate_comment_dayofyear|creationDate_comment_year|age|
+-----------+--------------------------+---------+------+--------------------+---------------+-----------------+----------+--------------------+-----------------------------+-------------------------------+------------------------------+------------------------------+-------------------------+---+
|  gmail.com|                         9|     7134|female|2019-03-20 04:12:...| 31.222.107.200|         

In [10]:
# 3. Count function of PySpark Dataframe

df.count()


41521

In [11]:
# 4. Statistical Properties of each column (not that interesting for this dataframe, misschien beter zelfs gewoon schrappen)

columns = ['gender', 'browserUsed', 'creationDate_comment_month']
for col in columns:
    print(col)
    df.describe([col]).show()


gender
+-------+------+
|summary|gender|
+-------+------+
|  count| 41521|
|   mean|  null|
| stddev|  null|
|    min|female|
|    max|  male|
+-------+------+

browserUsed
+-------+-----------+
|summary|browserUsed|
+-------+-----------+
|  count|      41521|
|   mean|       null|
| stddev|       null|
|    min|     Chrome|
|    max|     Safari|
+-------+-----------+

creationDate_comment_month
+-------+--------------------------+
|summary|creationDate_comment_month|
+-------+--------------------------+
|  count|                     41521|
|   mean|         7.688928494015077|
| stddev|        2.6296634895226774|
|    min|                         1|
|    max|                        11|
+-------+--------------------------+



# 5. Find unique values of a categorical column
columns = ['browserUsed', 'gender']
for x in columns:
    print(df.select(x).distinct().rdd.map(lambda r: r[0]).collect())



In [12]:
#7. Count the missing values in a column of PySpark Dataframe 


for col in df.columns:
    print(col, "with null values: ", df.filter(df[col].isNull()).count())
    

email with null values:  0
creationDate_comment_month with null values:  0
person_id with null values:  0
gender with null values:  0
creationDate_person with null values:  0
locationIP with null values:  0
browserUsed with null values:  0
comment_id with null values:  0
creationDate_comment with null values:  0
Number_of_comments_that_month with null values:  0
creationDate_comment_dayofmonth with null values:  0
creationDate_comment_dayofweek with null values:  0
creationDate_comment_dayofyear with null values:  0
creationDate_comment_year with null values:  0
age with null values:  0


In [13]:
df.write.save("data/Data_analysis_part1.parquet", mode='overwrite')


